# Train on TPU

In this notebook, we will use Nobrainer to train a model for brain extraction. Brain extraction is a common step in processing neuroimaging data. It is a voxel-wise, binary classification task, where each voxel is classified as brain or not brain. Incidentally, the name for the Nobrainer framework comes from creating models for brain extraction.

In the following cells, we will:

1. Get sample T1-weighted MR scans as features and FreeSurfer segmentations as labels.
    - We will binarize the FreeSurfer to get a precise brainmask.
2. Convert the data to TFRecords format.
3. Create two Datasets of the features and labels.
    - One dataset will be for training and the other will be for evaluation.
4. Instantiate a 3D convolutional neural network.
5. Choose a loss function and metrics to use.
6. Train on part of the data on TPU.
7. Evaluate on the rest of the data.

## Google Colaboratory

If you are using Colab, please switch your runtime to TPU. To do this, select `Runtime > Change runtime type` in the top menu. Then select TPU under `Hardware accelerator`.

In [ ]:
import nobrainer

# Get sample features and labels

We use 9 pairs of volumes for training and 1 pair of volumes for evaulation. Many more volumes would be required to train a model for any useful purpose.

In [ ]:
csv_of_filepaths = nobrainer.utils.get_data()
filepaths = nobrainer.io.read_csv(csv_of_filepaths)

train_paths = filepaths[:9]
evaluate_paths = filepaths[9:]

# Convert medical images to TFRecords

Remember how many full volumes are in the TFRecords files. This will be necessary to know how many steps are in on training epoch. The default training method needs to know this number, because Datasets don't always know how many items they contain.

In [ ]:
# Verify that all volumes have the same shape and that labels are integer-ish.

invalid = nobrainer.io.verify_features_labels(train_paths, num_parallel_calls=2)
assert not invalid

invalid = nobrainer.io.verify_features_labels(evaluate_paths)
assert not invalid

In [ ]:
!mkdir data

In [ ]:
# Convert training and evaluation data to TFRecords.

nobrainer.tfrecord.write(
    features_labels=train_paths,
    filename_template='data/data-train_shard-{shard:03d}.tfrec',
    examples_per_shard=3)

nobrainer.tfrecord.write(
    features_labels=evaluate_paths,
    filename_template='data/data-evaluate_shard-{shard:03d}.tfrec',
    examples_per_shard=1)

In [ ]:
!ls data

# Create Datasets

The batch size should be a multiple of 8.

In [ ]:
n_classes = 1
batch_size = 4
volume_shape = (256, 256, 256)
block_shape = (128, 128, 128)
n_epochs = None
augment = False
shuffle_buffer_size = 10
num_parallel_calls = 2

In [ ]:
dataset_train = nobrainer.dataset.get_dataset(
    file_pattern='data/data-train_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=n_epochs,
    augment=augment,
    shuffle_buffer_size=shuffle_buffer_size,
    num_parallel_calls=num_parallel_calls,
)

dataset_evaluate = nobrainer.dataset.get_dataset(
    file_pattern='data/data-evaluate_shard-*.tfrec',
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    block_shape=block_shape,
    n_epochs=1,
    augment=False,
    shuffle_buffer_size=None,
    num_parallel_calls=1,
)

In [ ]:
dataset_train

In [ ]:
dataset_evaluate

In [ ]:
# Get the steps for an epoch of training and an epoch of validation.
steps_per_epoch = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(train_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

validation_steps = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(evaluate_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

# Get TPU address and create TPU strategy

If you get an error that `COLAB_TPU_ADDR` is not found, you did not change your runtime to use TPU. Please do so by clicking `Runtime > Change runtime type` and select TPU under `Hardware accelerator`.

In [ ]:
import os
import tensorflow as tf

In [ ]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
strategy = tf.contrib.distribute.TPUStrategy(
    tpu_cluster_resolver=tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER),
)

# Instantiate and compile model within scope

In [ ]:
with strategy.scope():
    model = nobrainer.models.unet(n_classes=n_classes, input_shape=(*block_shape, 1), batch_size=batch_size)
    model.compile(
        optimizer=tf.train.AdamOptimizer(1e-04),
        loss=nobrainer.losses.jaccard,
        metrics=[nobrainer.metrics.dice])

In [ ]:
model.fit(
    dataset_train,
    epochs=5,
    steps_per_epoch=steps_per_epoch, 
    validation_data=dataset_evaluate, 
    validation_steps=validation_steps)

# Predict natively medical images (without TFRecords)

In [ ]:
# TODO